In [9]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

def crawl_url(url):
    """단일 URL 크롤링: AMP 먼저 요청, 없으면 일반 페이지 시도"""
    try:
        # 1. AMP 페이지 먼저 시도
        amp = url.rstrip("/") + "/amp"
        r = requests.get(amp, headers=HEADERS, timeout=12)

        # 2. AMP 없으면 일반 페이지 시도
        if r.status_code == 404:
            r = requests.get(url, headers=HEADERS, timeout=12)

        # 3. 여전히 에러면 None
        if r.status_code in (403, 404):
            return None
        r.raise_for_status()

        # 4. newspaper3k 파싱
        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()
        if not art.title or not art.text:
            return None

        return {
            "url": url,
            "title": art.title,
            "text": art.text,
            "publish_date": art.publish_date
        }
    except Exception:
        return None

def crawl_gen_articles_range(date_str, start_id, end_id, *,  # end_id는 '미포함' (파이썬 range 규칙)
                             max_workers=36, batch_size=1000, sleep_between=0.05,
                             save_path=None):
    """
    일반 기사 크롤링: [start_id, end_id) 구간만
      - AMP만 요청
      - 배치 제출
    """
    results, collected = [], 0
    total = end_id - start_id

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        with tqdm(total=total, desc=f"{date_str} [{start_id:05}-{end_id-1:05}]") as pbar:
            for start in range(start_id, end_id, batch_size):
                end = min(start + batch_size, end_id)
                urls = [f"{BASE_URL}{date_str}{num:05}" for num in range(start, end)]
                futures = [ex.submit(crawl_url, u) for u in urls]

                for fut in as_completed(futures):
                    res = fut.result()
                    if res:
                        results.append(res)
                        collected += 1
                    pbar.set_postfix_str(f"수집 {collected}")
                    pbar.update(1)
                    time.sleep(sleep_between)

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    if save_path:
        df.to_csv(save_path, index=False)
    tqdm.write(f"[{start_id:05}-{end_id-1:05}] 최종 수집: {len(df)}")
    return df

In [ ]:
day_str = "20250819"
start_id, end_id = 80000, 100000   # 이 부분만 각 노트북에서 바꿔서 실행
out_path = f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{day_str}_{start_id:05}-{end_id-1:05}.csv"

df_part = crawl_gen_articles_range(
    day_str, start_id, end_id,
    max_workers=36, batch_size=1000, sleep_between=0.05,
    save_path=out_path
)

df_part

20250819 [80000-99999]: 100%|██████████| 20000/20000 [21:59<00:00, 15.16it/s, 수집 329]


[80000-99999] 최종 수집: 329


,url,title,text,publish_date
0,https://www.hankyung.com/article/2025081980267,"[속보] 해병특검 ""내일 오후 1시 조태열 전 외교부 장관 소환""","특검, 조 전 장관 피의자 신분 조사 예정\n\n사진=연합뉴스\n\n채상병 사건 외...",2025-08-19 10:50:58+09:00
1,https://www.hankyung.com/article/2025081980296,"상승 출발 뒤 반락한 코스피, 3160선 후퇴",미·러·우 3자 회담 등 변수에 약세\n\n코스닥도 하락 전환\n\n19일 서울 중...,2025-08-19 11:14:20+09:00
2,https://www.hankyung.com/article/2025081980384,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
3,https://www.hankyung.com/article/2025081980437,"[속보] '극한가뭄' 강릉시, 20일 오전 9시부터 '제한급수' 돌입",수도 공급량 절반으로 제한\n\n지난 12일 오후 강원 강릉시 성산면 오봉리 오봉저...,2025-08-19 10:54:24+09:00
4,https://www.hankyung.com/article/2025081980507,"우크라 안전 보장에 한국도?…美국무 ""유럽 외 국가와도 협력""","美국무장관, 우크라 안보 구축에\n\n""비유럽국들과도 협력 의사"" 밝혀\n\n마코 ...",2025-08-19 11:09:28+09:00
...,...,...,...,...
324,https://www.hankyung.com/article/2025081993984,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
325,https://www.hankyung.com/article/2025081993994,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
326,https://www.hankyung.com/article/2025081994014,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
327,https://www.hankyung.com/article/2025081994024,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
